<a href="https://colab.research.google.com/github/tarunku/open_llm/blob/main/102_ReActAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install word2number

In [ ]:
!pip install -qU langchain langchain_community langchainhub langchain-openai langgraph

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = userdata.get("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = userdata.get("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=userdata.get('OPENAI_API_KEY'))


In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor
)

from langchain import hub
import inflect
import re

In [ ]:
def parsing_multiplier(string):
  numbers = re.findall(r'\d+', string )
  return int(numbers[0]), int(numbers[1])

a, b = parsing_multiplier('2, 4')

print(a)
print(b)

@tool(name_or_callable='next_num_tool')
def next_num(a:int) -> str:
    """get the next number."""
    return inflect.engine().number_to_words(int(a) + 12)


@tool(name_or_callable='addition_tool')
def add(input:str) -> str:
    """get sum of two numbers."""
    #a, b = parsing_multiplier(numbers)
    a, b = parsing_multiplier(input)
    return inflect.engine().number_to_words(int(a) + int(b))


In [ ]:
def agent_007(user_query: str):
  template_1 = '''
    You are a smart math assistant designed to compute the next number based on a given input, and sum of two numbers based on given inoouts.

    You have access to the following tool:

    **next_num(a: int):** Returns the next number by adding 12 to the given input and converting it to words.
    **add(a: int, b: int):** Returns the sum of given numbers in input and converting it to words.

    When given a user query, determine whether it requires this tool. If it does, use appropriate tool to perform the calculation and return only the final numeric result.

    If the available tool is not sufficient to answer the query, respond with: "I can't handle this query."

    -----------
    Now here is the user query: {user_query}
    '''

  template = '''
    You are a smart math assistant capable of performing two specific calculations:

    - **next_num(a: int):** Adds 12 to the given input and returns the result in words.
    - **add(a: int, b: int):** Computes the sum of two numbers and returns the result in words.

    When given a user query:
    - Identify whether it requires one of the available tools.
    - If so, use the appropriate tool to compute the result and return **only** the final numeric output.
    - If the tools are insufficient to answer the query, respond with: **"I can't handle this query."**

    -----------
    User Query: {user_query}
    '''

  prompt_template = PromptTemplate(
        template=template, input_variables=["user_query"]
    )

  react_prompt = hub.pull("hwchase17/react")

  tools = [next_num, add]  # Ensure tools are correctly referenced

  agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
  agent_executor = AgentExecutor(
        agent=agent, # Pass the agent instance here
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=10,
        max_execution_time=60
    )
  result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(user_query=user_query)}
    )

  return result["output"]



In [ ]:
agent_007('next of next of sum of 4 and 6')

In [ ]:
agent_007('sum of next of 10 and next of sum of 4 and 6')

In [ ]:
import re
from word2number import w2n

def extract_numbers(text):
    """Extract both numeric and word-based numbers from a string and return them as a list of integers."""
    # Extract numeric values using regex
    numeric_numbers = re.findall(r'-?\d+(\.\d+)?', text)
    numeric_numbers = [float(num) if '.' in num else int(num) for num in numeric_numbers]

    # Extract word-based numbers
    words = text.lower().split()
    word_numbers = []

    for word in words:
        try:
            word_numbers.append(w2n.word_to_num(word))
        except ValueError:
            continue  # Skip words that aren't numbers

    # Combine and return all numbers
    return numeric_numbers + word_numbers


def parse_multipliers(text):
    """Extract exactly two numbers from the text (numeric or word-based) for operations like multiplication."""
    numbers = extract_numbers(text)

    if len(numbers) < 2:
        raise ValueError("Not enough numbers found in the text for parsing multipliers.")

    return numbers[0], numbers[1]  # Return the first two numbers found


# ✅ Example Usage:
text = "Multiply twenty-two by 4"
print(parse_multipliers(text))  # Output: (22, 4)

text2 = "Find the product of seven and 3"
print(parse_multipliers(text2))  # Output: (7, 3)


In [ ]:
extract_numbers("twenty-two and twenty-two")